In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32986
2,Huelva,Confirmados PDIA 14 días,1157
3,Huelva,Tasa PDIA 14 días,"225,4613480912758"
4,Huelva,Confirmados PDIA 7 días,472
5,Huelva,Total Confirmados,33169
6,Huelva,Curados,30941
7,Huelva,Fallecidos,388


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32986.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9021.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 817 personas en los últimos 7 días 

Un positivo PCR cada 289 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32986.0,472.0,1157.0,513170.0,91.977317,225.461348,173.0
Huelva-Costa,19006.0,255.0,699.0,289548.0,88.068300,241.410751,104.0
Huelva (capital),9021.0,176.0,498.0,143837.0,122.360728,346.225241,73.0
Condado-Campiña,10710.0,167.0,354.0,156231.0,106.892998,226.587553,56.0
Sierra de Huelva-Andévalo Central,2917.0,48.0,101.0,67391.0,71.226128,149.871645,12.0
Moguer,1378.0,27.0,67.0,21867.0,123.473728,306.397768,12.0
Beas,232.0,17.0,21.0,4341.0,391.614835,483.759502,10.0
Almonte,1905.0,40.0,59.0,24507.0,163.218672,240.747542,8.0
Cartaya,1380.0,17.0,30.0,20083.0,84.648708,149.380073,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,141.0,8.0,20.0,3092.0,258.732212,646.830530,2.0
Beas,232.0,17.0,21.0,4341.0,391.614835,483.759502,10.0
San Juan del Puerto,566.0,14.0,43.0,9411.0,148.762087,456.912124,6.0
Valverde del Camino,657.0,30.0,58.0,12750.0,235.294118,454.901961,6.0
Lucena del Puerto,267.0,7.0,12.0,3261.0,214.658080,367.985281,2.0
Calañas,201.0,6.0,10.0,2768.0,216.763006,361.271676,1.0
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0
Huelva (capital),9021.0,176.0,498.0,143837.0,122.360728,346.225241,73.0
Moguer,1378.0,27.0,67.0,21867.0,123.473728,306.397768,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,179.0,1.0,11.0,3933.0,25.425884,279.684719,0.0,0.090909
Aljaraque,1160.0,8.0,39.0,21474.0,37.254354,181.614976,3.0,0.205128
Aracena,539.0,2.0,9.0,8255.0,24.227741,109.024833,0.0,0.222222
Ayamonte,1332.0,3.0,11.0,21104.0,14.215315,52.122820,1.0,0.272727
San Juan del Puerto,566.0,14.0,43.0,9411.0,148.762087,456.912124,6.0,0.325581
Villanueva de los Castillejos,165.0,1.0,3.0,2825.0,35.398230,106.194690,0.0,0.333333
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0,0.333333
Palos de la Frontera,707.0,9.0,26.0,11742.0,76.647931,221.427355,1.0,0.346154
Huelva (capital),9021.0,176.0,498.0,143837.0,122.360728,346.225241,73.0,0.353414
